In [4]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline
import math
import os
#os.chdir("C:/Users/ACER/anaconda3/data_analys/Lect_04/задача_2_Economics of Cities/")

from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [50]:
pd.set_option('display.max_columns', None)
original_df = pd.read_csv("930941248_T_ONTIME_REPORTING.csv", sep=",", decimal='.')
original_df

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,DEST,DEST_CITY_NAME,DEST_STATE_NM,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 33
0,2020,8,24,1,WN,N8680C,1558,DEN,"Denver, CO",Colorado,DSM,"Des Moines, IA",Iowa,1345,1341.0,-4.0,9.0,17.0,1625,1623.0,-2.0,0.0,NaN,0.0,100.0,102.0,76.0,589.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,8,24,1,WN,N7811F,1596,DSM,"Des Moines, IA",Iowa,DEN,"Denver, CO",Colorado,1230,1222.0,-8.0,7.0,9.0,1325,1258.0,-27.0,0.0,NaN,0.0,115.0,96.0,80.0,589.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2020,8,24,1,WN,N963WN,450,DSM,"Des Moines, IA",Iowa,LAS,"Las Vegas, NV",Nevada,1545,1620.0,35.0,12.0,5.0,1655,1707.0,12.0,0.0,NaN,0.0,190.0,167.0,150.0,1216.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,8,24,1,WN,N915WN,374,DSM,"Des Moines, IA",Iowa,STL,"St. Louis, MO",Missouri,700,701.0,1.0,12.0,3.0,800,800.0,0.0,0.0,NaN,0.0,60.0,59.0,44.0,259.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,8,24,1,WN,N8680C,1558,DSM,"Des Moines, IA",Iowa,STL,"St. Louis, MO",Missouri,1715,1707.0,-8.0,5.0,4.0,1820,1758.0,-22.0,0.0,NaN,0.0,65.0,51.0,42.0,259.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3367,2020,8,24,1,UA,N831UA,433,DEN,"Denver, CO",Colorado,DSM,"Des Moines, IA",Iowa,1750,1746.0,-4.0,16.0,4.0,2041,2023.0,-18.0,0.0,NaN,0.0,111.0,97.0,77.0,589.0,NaN,NaN,NaN,NaN,NaN,NaN
3368,2020,8,23,7,UA,N822UA,771,DSM,"Des Moines, IA",Iowa,DEN,"Denver, CO",Colorado,753,832.0,39.0,9.0,7.0,841,911.0,30.0,0.0,NaN,0.0,108.0,99.0,83.0,589.0,30.0,0.0,0.0,0.0,0.0,NaN
3369,2020,8,23,7,UA,N827UA,433,DEN,"Denver, CO",Colorado,DSM,"Des Moines, IA",Iowa,1750,1742.0,-8.0,16.0,3.0,2041,2017.0,-24.0,0.0,NaN,0.0,111.0,95.0,76.0,589.0,NaN,NaN,NaN,NaN,NaN,NaN
3370,2020,8,22,6,UA,N851UA,771,DSM,"Des Moines, IA",Iowa,DEN,"Denver, CO",Colorado,753,744.0,-9.0,11.0,8.0,841,824.0,-17.0,0.0,NaN,0.0,108.0,100.0,81.0,589.0,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
df = original_df[[
 'DAY_OF_MONTH',
 'DAY_OF_WEEK',
 'OP_UNIQUE_CARRIER',
 'ORIGIN',
 'ORIGIN_STATE_NM',
 'DEST',
 'DEST_CITY_NAME',
 'DEST_STATE_NM',
 'CRS_DEP_TIME',
 'DEP_DELAY',
 'DISTANCE']]

# Фильтрация по DEP_DELAY
df = df[(df['DEP_DELAY'] != 0) & df['DEP_DELAY'].between(-40, 60)]

# Выбираем только один штат
df = df[df.ORIGIN_STATE_NM == 'Iowa'] # size = 1685
df = df.drop(columns=['DEST_CITY_NAME', 'DEST_STATE_NM'])

# Конвертируем время hhmm в минуты
time = df['CRS_DEP_TIME'].copy()
df['CRS_DEP_TIME'] = time % 100 + 60 * (time // 100)

df.agg({'DEP_DELAY': ['min', 'max', 'mean', 'median']}) #оценка параметров

,DEP_DELAY
min,-37.000000
max,59.000000
mean,-5.222009
median,-7.000000


In [86]:
prep_df = df[[
 'DAY_OF_MONTH',
 'DAY_OF_WEEK',
 'CRS_DEP_TIME',
 'DEP_DELAY',
 'DISTANCE']]
prep_df.head()

,DAY_OF_MONTH,DAY_OF_WEEK,CRS_DEP_TIME,DEP_DELAY,DISTANCE
1,24,1,750,-8.0,589.0
2,24,1,945,35.0,1216.0
3,24,1,420,1.0,259.0
4,24,1,1035,-8.0,259.0
9,25,2,750,-9.0,589.0


In [91]:
carriers = df["OP_UNIQUE_CARRIER"].unique()
print(carriers)
cities = df["ORIGIN"].unique()
print(cities)

prep_df[carriers] = 0

for carrier in carriers:
    prep_df.loc[data_frame['OP_UNIQUE_CARRIER'] == carrier, carrier] = 1

# перемешивание данных
prep_df = prep_df.sample(frac=1) 

['WN' '9E' 'MQ' 'OO' 'UA' 'YV' 'AA' 'YX' 'F9' 'G4' 'OH']
['DSM' 'CID' 'ALO' 'DBQ' 'SUX']


In [139]:
#for col in df.columns: if sum(df[col].isnull())/float(len(df.index)) > 0.8: del df[col] - удалить с процентом nan

X = prep_df.drop(columns=["DEP_DELAY"])
Y = prep_df['DEP_DELAY']
np.append(X.iloc[1].values, 123)
print(X)
b = [i + 1 for i in range(X.shape[1])]
print((((X * b - b).sum(axis=1) - Y) ** 2).sum())

      DAY_OF_MONTH  DAY_OF_WEEK  CRS_DEP_TIME  DISTANCE  WN  9E  MQ  OO  UA  \
1180             6            4           494     147.0   0   0   1   0   0   
407             20            4           722     685.0   0   0   1   0   0   
513             21            5           693     147.0   0   0   1   0   0   
444              5            3           772     234.0   0   0   1   0   0   
1640            24            1          1090     589.0   0   0   0   0   0   
...            ...          ...           ...       ...  ..  ..  ..  ..  ..   
1286            22            6          1056     589.0   0   0   0   1   0   
343             14            5           360     232.0   0   1   0   0   0   
226             21            5          1050     220.0   0   1   0   0   0   
150              7            5           960     743.0   0   1   0   0   0   
2122             1            6           668     232.0   0   1   0   0   0   

      YV  AA  YX  F9  G4  OH  
1180   0   0   0   0

,DAY_OF_MONTH,DAY_OF_WEEK,CRS_DEP_TIME,DISTANCE,WN,9E,MQ,OO,UA,YV,AA,YX,F9,G4,OH
1180,5,6,1479,584.0,-5,-6,0,-8,-9,-10,-11,-12,-13,-14,-15
407,19,6,2163,2736.0,-5,-6,0,-8,-9,-10,-11,-12,-13,-14,-15
513,20,8,2076,584.0,-5,-6,0,-8,-9,-10,-11,-12,-13,-14,-15
444,4,4,2313,932.0,-5,-6,0,-8,-9,-10,-11,-12,-13,-14,-15
1640,23,0,3267,2352.0,-5,-6,-7,-8,-9,-10,-11,-12,0,-14,-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1286,21,10,3165,2352.0,-5,-6,-7,0,-9,-10,-11,-12,-13,-14,-15
343,13,8,1077,924.0,-5,0,-7,-8,-9,-10,-11,-12,-13,-14,-15
226,20,8,3147,876.0,-5,0,-7,-8,-9,-10,-11,-12,-13,-14,-15
150,6,8,2877,2968.0,-5,0,-7,-8,-9,-10,-11,-12,-13,-14,-15


In [93]:
# нормируем колонки по максимуму
X = X / X.aggregate("max")

In [274]:
import numpy as np

def prediction(x, b):
    return np.dot(np.append(x, 1), b)

def calculate_error(X, Y, b):
    Y_mean = Y.mean()
    pred = (X * b).sum(axis=1) + Y_mean
    return ((Y - pred - Y_mean) ** 2).sum()

def calculate_error_with_regul(X, Y, b, reg_par):
    regul_error = reg_par * (b ** 2 + abs(b))
    pred = (X * b + regul_error).sum(axis=1)
    return ((Y - pred - Y.mean()) ** 2).sum() + regul_error.sum()

def gradient(X,Y, b):
    h = 0.00001
    Error = calculate_error(X,Y, b)
    size = len(b)
    grad = np.zeros(size)
    for i in range(size):
        b[i] += h
        #grad[i] = (Error - calculate_error_by_Kfold(X,Y, b))/h
        grad[i] = (Error - calculate_error(X,Y, b))
        b[i] -= h
    return grad, Error

def gradient_with_regul(X, Y, b, reg_par):
    h = 0.00001
    Error = calculate_error_with_regul(X, Y, b, reg_par)
    size = len(b)
    grad = np.zeros(size)
    for i in range(size):
        b[i] += h
        grad[i] = (Error - calculate_error_with_regul(X,Y, b, reg_par))
        b[i] -= h
    return grad, Error  

    
def SGD(X, Y, b, reg_par = 0, num_iter=50):
    print('NEW')
    coef_grad = 1
    coef_prev_grad = 0.2
    prev_grad, prev_error = gradient_with_regul(X,Y,b, reg_par)
    b = b + prev_grad * coef_grad
    error = prev_error + 1
    next_error = 0
    for c in range(1, num_iter + 1):
        if abs(error - prev_error) <= 0.001: break
        grad, error = gradient_with_regul(X,Y,b, reg_par)
        b = b + grad * coef_grad + prev_grad * coef_prev_grad
        next_error = calculate_error_with_regul(X,Y, b, reg_par)
        # проверка действительно ли следующий шаг норм и смена коеффициентов
        while next_error > error:
            coef_grad /= 2
            coef_prev_grad /= 2
            b = b - grad * coef_grad + prev_grad * coef_prev_grad
            next_error = calculate_error_with_regul(X,Y, b, reg_par)
        else:
            coef_grad *= 2
            coef_prev_grad *= 2
            
        prev_grad = grad
        prev_error = error
        error = next_error
        print(f'{c}  :  {error=}')
        
    return b, calculate_error(X, Y, b)


def Adagrad(X, Y, b, reg_par = 0, num_iter = 50):
    print('NEW')
    coef_grad = 0.1
    G_vector = np.zeros(len(b)) + 0.01
    G_vector_sqr = G_vector ** 2
    
    next_error = 0
    for c in range(1, num_iter + 1):
        grad, error = gradient_with_regul(X,Y,b, reg_par)
        G_vector_sqr = G_vector_sqr + grad ** 2
        b = b + (grad * coef_grad) / G_vector
        
        next_error = calculate_error_with_regul(X,Y, b, reg_par)
        # проверка действительно ли следующий шаг норм и смена коеффициентов
        while next_error > error:
            coef_grad /= 2
            b = b - (grad * coef_grad) / G_vector
            next_error = calculate_error_with_regul(X,Y, b, reg_par)
        else:
            coef_grad *= 2
        G_vector = G_vector_sqr ** 0.5
        error = next_error
        print(f'{c}  :  {error=};  {coef_grad=}')
    return b, calculate_error(X, Y, b)


def RMSprop(X, Y, b, scale_coef=0.5, reg_par = 0, num_iter = 50):
    print('NEW')
    coef_grad = 0.1
    G_vector = np.zeros(len(b)) + 0.01
    G_vector_sqr = G_vector ** 2

    next_error = 0
    for c in range(1, num_iter + 1):
        grad, error = gradient_with_regul(X, Y, b, reg_par)
        
        G_vector_sqr = G_vector_sqr * scale_coef + (1 - scale_coef) * grad ** 2
        b = b + (grad * coef_grad) / G_vector
        
        next_error = calculate_error_with_regul(X, Y, b, reg_par)
        # проверка действительно ли следующий шаг норм и смена коеффициентов
        while (next_error := calculate_error_with_regul(X, Y, b, reg_par)) > error:
            coef_grad /= 2
            b = b - (grad * coef_grad) / G_vector
        else:
            coef_grad *= 2

        G_vector = G_vector_sqr ** 0.5
        error = next_error
        print(f'{c}  :  {error=};  {coef_grad=}')
    return b, calculate_error(X, Y, b)


def Adam(X, Y, b, scale_coef_M = 0.5, scale_coef_G = 0.5, reg_par = 0, num_iter = 50):
    # здесь G_vector - то же самое что G_vector_sqr в прошлыых двух
    print('NEW')
    coef_grad = 0.1
    G_vector = np.zeros(len(b)) + 0.01
    M_vector = np.zeros(len(b))
    
    next_error = 0
    for c in range(1, num_iter + 1):
        grad, error = gradient_with_regul(X, Y, b, reg_par)
        
        M_vector = M_vector * scale_coef_M + (1 - scale_coef_M) * grad
        G_vector = G_vector * scale_coef_G + (1 - scale_coef_G) * grad ** 2
        
        m_ = M_vector / (1 - scale_coef_M ** c)
        g_ = (G_vector / (1 - scale_coef_G ** c)) ** 0.5
        b = b + m_ * coef_grad / g_
        
        next_error = calculate_error_with_regul(X,Y, b, reg_par)
        # проверка действительно ли следующий шаг норм и смена коеффициентов
        while next_error > error:
            coef_grad /= 2
            m_ = M_vector / (1 - scale_coef_M ** c)
            g_ = (G_vector / (1 - scale_coef_G ** c)) ** 0.5 
            b = b - m_ * coef_grad / g_
      
            next_error = calculate_error_with_regul(X, Y, b, reg_par)
            print(next_error, error, coef_grad)
        else:
            coef_grad *= 2
        error = next_error
        print(f'{c}  :  {error=};  {coef_grad=}')
    return b, calculate_error(X, Y, b)


def K_fold_by_reg_param(X, Y, b, reg_params):
    size_test = len(X)//len(reg_params)
    begin = 0
    end = size_test + len(X)%len(reg_params)
    best_b=[]
    best_error = 1000000000# не знаю как поставить бесконечность
    best_reg_param = reg_params[0]
    for i in range(0, len(reg_params)):
        
        X_train = pd.concat([X.iloc[0:begin],X.iloc[end:len(X)]])
        Y_train = pd.concat([Y.iloc[0:begin],Y.iloc[end:len(Y)]])
        b, error = SGD(X_train, Y_train, b, reg_params[i])
        
        error = calculate_error(X.iloc[begin:end],Y.iloc[begin:end], b)
        if error < best_error:
            best_error = error
            best_b = b
            best_reg_param = reg_params[i]
            
        begin += size_test
        end += size_test
        
    return best_b, best_error, best_reg_param

In [276]:
def prediction(x, b):
    res = 0
    for i in range(0, len(x)):
        res += x[i]*b[i]
    return res+b[-1]

def difference(y, y_):
    return abs(y-y_)

def calculate_error(X,Y, b):
    error=0
    for i in range(0, len(X)):
        x = X.iloc[i].to_list()
        y = Y.iloc[i]
        #error +=  difference(y, prediction(x, b))**2
        pred = 0
        for i in range(0, len(x)):
            pred += x[i]*b[i]
        error += (y-pred-b[-1])**2
    return error

def calculate_error_with_regul(X,Y, b, reg_par):
    error=0
    for i in range(0, len(X)):
        x = X.iloc[i].to_list()
        y = Y.iloc[i]
        #error +=  difference(y, prediction(x, b))**2
        pred = 0
        regul_error = 0
        for i in range(0, len(x)):
            pred += x[i]*b[i]+reg_par*(b[i]**2+abs(b[i]))
            regul_error += b[i]**2+abs(b[i])
        error += (y-pred-b[-1])**2 + reg_par*regul_error
    return error


def gradient(X,Y, b):
    h = 0.00001
    Error = calculate_error(X,Y, b)
    size = len(b)
    grad = [0 for i in range(0, size)]
    for i in range(0, size):
        b[i] += h
        #grad[i] = (Error - calculate_error_by_Kfold(X,Y, b))/h
        grad[i] = (Error - calculate_error(X,Y, b))
        b[i] -= h
    return grad, Error

def gradient_with_regul(X,Y, b,reg_par):
    h = 0.00001
    Error = calculate_error_with_regul(X,Y, b, reg_par)
    size = len(b)
    grad = [0 for i in range(0, size)]
    for i in range(0, size):
        b[i] += h
        grad[i] = (Error - calculate_error_with_regul(X,Y, b, reg_par))
        b[i] -= h
    return grad, Error  

    
def SGD(X,Y, b, reg_par = 0):
    coef_grad = 1
    coef_prev_grad = 0.2
    prev_grad, prev_error =gradient_with_regul(X,Y,b, reg_par)
    for i in range(0, len(b)):
        b[i]+= prev_grad[i] * coef_grad
    c = 0
    error = prev_error+ 1
    next_error = 0
    while (abs(error - prev_error)> 0.001) & (c < 50):
        grad, error = gradient_with_regul(X,Y,b, reg_par)
        c+=1
        
        for i in range(0, len(b)):
            b[i]+= grad[i]*coef_grad + prev_grad[i] * coef_prev_grad
        next_error = calculate_error_with_regul(X,Y, b, reg_par)
        # проверка действительно ли следующий шаг норм и смена коеффициентов
        while_was_not_activated = True
        while next_error > error:
            coef_grad /= 2
            coef_prev_grad /= 2
            for i in range(0, len(b)):
                b[i] -= grad[i]*coef_grad + prev_grad[i] * coef_prev_grad
            next_error = calculate_error_with_regul(X,Y, b, reg_par)
            while_was_not_activated = False
        if while_was_not_activated:
            coef_grad *= 2
            coef_prev_grad *= 2
            
        prev_grad = grad
        prev_error = error
        error = next_error
        print(c, ' :', error)
    return b, calculate_error(X,Y, b)


def Adagrad(X,Y, b, reg_par = 0):
    coef_grad = 0.1
    G_vector = [0.01 for i in range(0, len(b))]
    G_vector_sqr = [i**2 for i in G_vector]
    
    c = 0
    next_error = 0
    while c < 50:
        grad, error = gradient_with_regul(X,Y,b, reg_par)
        c+=1
        for i in range(0, len(b)):
            G_vector_sqr[i] += grad[i]**2
            b[i]+= grad[i]*coef_grad/G_vector[i]
        
        
        next_error = calculate_error_with_regul(X,Y, b, reg_par)
        # проверка действительно ли следующий шаг норм и смена коеффициентов
        while_was_not_activated = True
        while next_error > error:
            coef_grad /= 2
            for i in range(0, len(b)):
                b[i] -= grad[i]*coef_grad/G_vector[i]
            next_error = calculate_error_with_regul(X,Y, b, reg_par)
            while_was_not_activated = False
        if while_was_not_activated:
            coef_grad *= 2
        for i in range(0, len(b)):
            G_vector[i] = G_vector_sqr[i]**0.5
        error = next_error
        print(c, ' :', error, '; coef:', coef_grad)
    return b, calculate_error(X,Y, b)


def RMSprop(X, Y, b, scale_coef=0.5, reg_par = 0):
    coef_grad = 0.1
    G_vector = [0.01 for i in range(0, len(b))]
    G_vector_sqr = [i**2 for i in G_vector]
    
    c = 0
    next_error = 0
    while c < 50:
        grad, error = gradient_with_regul(X, Y, b, reg_par)
        c+=1
        for i in range(0, len(b)):
            G_vector_sqr[i] = G_vector_sqr[i] * scale_coef +  (1-scale_coef) * grad[i]**2
            b[i] += grad[i]*coef_grad/G_vector[i]
        
        
        next_error = calculate_error_with_regul(X, Y, b, reg_par)
        # проверка действительно ли следующий шаг норм и смена коеффициентов
        while_was_not_activated = True
        while next_error > error:
            coef_grad /= 2
            for i in range(0, len(b)):
                b[i] -= grad[i]*coef_grad/G_vector[i]
            next_error = calculate_error_with_regul(X, Y, b, reg_par)
            while_was_not_activated = False
        if while_was_not_activated:
            coef_grad *= 2
        for i in range(0, len(b)):
            G_vector[i] = G_vector_sqr[i]**0.5
        error = next_error
        print(c, ' :', error, '; coef:', coef_grad)
    return b, calculate_error(X,Y, b)


def Adam(X,Y, b, scale_coef_M=0.5, scale_coef_G=0.5, reg_par = 0):
    # здесь G_vector - то же самое что G_vector_sqr в прошлыых двух
    coef_grad = 0.1
    G_vector = [0.01 for i in range(0, len(b))]
    
    M_vector = [0 for i in range(0, len(b))]
    
    c = 0
    next_error = 0
    while c < 50:
        grad, error = gradient_with_regul(X,Y,b, reg_par)
        c+=1
        for i in range(0, len(b)):
            M_vector[i] = M_vector[i] * scale_coef_M + (1-scale_coef_M)* grad[i]
            G_vector[i] = G_vector[i] * scale_coef_G + (1-scale_coef_G)* grad[i]**2
            
            m_ = M_vector[i]/(1-scale_coef_M**c)
            g_ = (G_vector[i]/ (1-scale_coef_G**c))**0.5
            
            b[i]+= m_*coef_grad/g_
        
        
        next_error = calculate_error_with_regul(X,Y, b, reg_par)
        # проверка действительно ли следующий шаг норм и смена коеффициентов
        while_was_not_activated = True
        while next_error > error:
            coef_grad /= 2
            for i in range(0, len(b)):
                m_ = M_vector[i]/(1-scale_coef_M**c)
                g_ = (G_vector[i]/ (1-scale_coef_G**c))**0.5
                b[i]-= m_*coef_grad/g_
                
            next_error = calculate_error_with_regul(X,Y, b, reg_par)
            while_was_not_activated = False
            print(next_error, error, coef_grad)
        if while_was_not_activated:
            coef_grad *= 2
        error = next_error
        print(c, ' :', error, '; coef:', coef_grad)
    return b, calculate_error(X,Y, b)


def K_fold_by_reg_param(X, Y, b, reg_params):
    size_test = len(X)//len(reg_params)
    begin = 0
    end = size_test + len(X)%len(reg_params)
    best_b=[]
    best_error = 1000000000# не знаю как поставить бесконечность
    best_reg_param = reg_params[0]
    for i in range(0, len(reg_params)):
        
        X_train = pd.concat([X.iloc[0:begin],X.iloc[end:len(X)]])
        Y_train = pd.concat([Y.iloc[0:begin],Y.iloc[end:len(Y)]])
        b, error = SGD(X_train, Y_train, b, reg_params[i])
        
        error = calculate_error(X.iloc[begin:end],Y.iloc[begin:end], b)
        if error < best_error:
            best_error = error
            best_b = b
            best_reg_param = reg_params[i]
            
        begin += size_test
        end += size_test
        
    return best_b, best_error, best_reg_param

In [267]:
b = np.ones(len(columns))
RMSprop(X, Y, b, scale_coef=0.8, reg_par=1)
# 3634581 - maen

NEW
1  :  error=10811865.360255165;  coef_grad=7.62939453125e-07
2  :  error=10811835.234505236;  coef_grad=1.52587890625e-06
3  :  error=10811768.0032536;  coef_grad=3.0517578125e-06
4  :  error=10811618.03660645;  coef_grad=6.103515625e-06
5  :  error=10811283.721302412;  coef_grad=1.220703125e-05
6  :  error=10810539.007396575;  coef_grad=2.44140625e-05
7  :  error=10808881.684448492;  coef_grad=4.8828125e-05
8  :  error=10805197.93772237;  coef_grad=9.765625e-05
9  :  error=10797023.420666473;  coef_grad=0.0001953125
10  :  error=10778924.834784966;  coef_grad=0.000390625
11  :  error=10738990.229774412;  coef_grad=0.00078125
12  :  error=10651360.005291494;  coef_grad=0.0015625
13  :  error=10460942.195920961;  coef_grad=0.003125
14  :  error=10054831.46647415;  coef_grad=0.00625
15  :  error=9220856.161710892;  coef_grad=0.0125
16  :  error=7640214.918716096;  coef_grad=0.025
17  :  error=5134503.833475588;  coef_grad=0.05
18  :  error=2554664.598414531;  coef_grad=0.1
19  :  err

(array([ 0.7003164 ,  0.85308798, -0.03695045, -0.01658877,  0.88808998,
         0.88990539,  0.87521638,  0.87585823,  0.87628719,  0.87574567,
         0.88353905,  0.86935646,  0.88760114,  0.88737454,  0.8782163 ]),
 1369304.3507188633)

In [277]:
b = np.array([0.07108979547236088,
 -0.9733133421334863,
 0.9968873042940036,
 4.109941361898274,
 2.059924274732903,
 0.7515062699570404,
 -2.3957530262211173,
 -1.5086747078487333,
 -4.630649158547469,
 -1.84807660184145,
 -1.8403365958128066,
 -2.8559587160828506,
 2.721951107553011,
 -9.714869260404695,
 -2.1713094116244145])
#b = np.ones(len(columns))
Adam(X, Y, b, scale_coef_M=Y.mean(), scale_coef_G=0.1, reg_par=0.8)
# 3634581 - maen

1  : 16702743275.294611 ; coef: 0.2
2  : 13882875504.930542 ; coef: 0.4
3  : 8704786152.303185 ; coef: 0.8
4  : 1394057334.0000682 ; coef: 1.6
12266661845.894236 1394057334.0000682 0.8
2343709183.545454 1394057334.0000682 0.4
663564181.087439 1394057334.0000682 0.2
5  : 663564181.087439 ; coef: 0.2
3065393310.1810412 663564181.087439 0.1
1397869938.8918803 663564181.087439 0.05
911626985.8606566 663564181.087439 0.025
757516277.1754427 663564181.087439 0.0125
702981925.7290627 663564181.087439 0.00625
681378659.931883 663564181.087439 0.003125
671997219.4359524 663564181.087439 0.0015625
667662074.6238546 663564181.087439 0.00078125
665583462.0230516 663564181.087439 0.000390625
664566403.9190824 663564181.087439 0.0001953125
664063437.9469597 663564181.087439 9.765625e-05
663813345.8597177 663564181.087439 4.8828125e-05
663688647.5569066 663564181.087439 2.44140625e-05
663626385.342716 663564181.087439 1.220703125e-05
663595275.970178 663564181.087439 6.103515625e-06
663579726.7175798

663564181.0874394 663564181.087439 3.587324068671532e-44
663564181.0874394 663564181.087439 1.793662034335766e-44
663564181.0874394 663564181.087439 8.96831017167883e-45
663564181.0874394 663564181.087439 4.484155085839415e-45
663564181.0874394 663564181.087439 2.2420775429197074e-45
663564181.0874394 663564181.087439 1.1210387714598537e-45
663564181.0874394 663564181.087439 5.6051938572992686e-46
663564181.0874394 663564181.087439 2.8025969286496343e-46
663564181.0874394 663564181.087439 1.4012984643248171e-46
663564181.0874394 663564181.087439 7.006492321624086e-47
663564181.0874394 663564181.087439 3.503246160812043e-47
663564181.0874394 663564181.087439 1.7516230804060214e-47
663564181.0874394 663564181.087439 8.758115402030107e-48
663564181.0874394 663564181.087439 4.3790577010150536e-48
663564181.0874394 663564181.087439 2.1895288505075268e-48
663564181.0874394 663564181.087439 1.0947644252537634e-48
663564181.0874394 663564181.087439 5.473822126268817e-49
663564181.0874394 66356

KeyboardInterrupt: 

In [78]:
b = [0.07108979547236088,
 -0.9733133421334863,
 0.9968873042940036,
 4.109941361898274,
 2.059924274732903,
 0.7515062699570404,
 -2.3957530262211173,
 -1.5086747078487333,
 -4.630649158547469,
 -1.84807660184145,
 -1.8403365958128066,
 -2.8559587160828506,
 2.721951107553011,
 -9.714869260404695,
 -2.1713094116244145,
 -4.869790876994323]
error=0
for i in range(0, len(X)):
    x = X.iloc[i].to_list()
    y = Y.iloc[i]
    #error +=  difference(y, prediction(x, b, g))**2
    pred = 0
    for i in range(0, len(x)):
        pred += x[i]*b[i]
    error += abs(y-pred-b[-1])
(error/len(X))

5.28897812869417

In [79]:
error=0
for i in range(0, len(X)):
    y = Y.iloc[i]
    error += abs(y-g)
error / len(X)

9.287431294945453

In [55]:
prep_df.agg({'CRS_DEP_TIME': 'min' })[0]

0.5120910384068279

In [ ]:
last_b = [-0.030386120488803316,
 -0.029674049015335522,
 -0.0035100154031383105,
 -0.003200133522719021,
 -0.001445508196131786,
 0.0012666197926167317,
 -0.014518818537591143,
 -0.013747216772204616,
 -0.004716126209607403,
 -0.016204393326723784,
 -0.010267026861892499,
 0.004420402418926793,
 0.0033827350743404943,
 -0.01311095885014846,
 0.009417268801753656] # на -20...20

b = [-1.0916733579732403,
 -2.5640838101518204,
 2.24665028274803,
 3.4670553823936916,
 2.311274230184649,
 1.2462326201785583,
 -2.147353830572917,
 -1.0697943903179663,
 -2.2125041864212505,
 -2.023801340550869,
 -1.373493009975614,
 -2.3276501602169244,
 2.156534080207264,
 -7.107128976965588,
 -2.2446902880543216,
 -3.4282117381240815] # -40...90

In [82]:
Y.agg({'DEP_DELAY': 'mean' })[0]
pd.concat([data_frame.iloc[0:10],data_frame.iloc[20:30]]).head(20)

-5.222008957133717